# 0. Imports

In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import matplotlib.gridspec as gridspec

from ydata_profiling import ProfileReport
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from kneed import KneeLocator
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from collections import Counter
from tabulate import tabulate
from datetime import datetime, timedelta

# I. Data Pre-processing

In [2]:
os.getcwd()

'c:\\Users\\MaximeRoux\\bright_river'

In [3]:
# Configure seaborn plot style: set background color and use dark grid
sns.set(rc={'axes.facecolor':'#E6E6E6'}, style='darkgrid')

In [4]:
# Importing the dataset
df = pd.read_csv('data/sales_invoiced_lines_basics_v2.csv')

In [5]:
# The dataframe is relatively large and we want to visualise it as a scrollable element
pd.set_option('display.max_columns', None)

## 1. Data cleaning

In [6]:
df.head()

,Unnamed: 0,Id,DimEntityId,DimOrderTypeId,OrderTypeCode,DimGeographyId,Country,DimPostingDateId,DimGenderId,DimBillToCustomerId,CustomerMainType,CustomerSubGroup,DimCollectionId,DimItemId,ItemNo2,StyleID,Style,Color,Size,EAN,Gender,Category,SUB Category,SUB Category2,Quality,Origin Country,WHSSalesPriceDKK,RRSalesPriceDKK,DimOrderSalespersonId,DimOrderCountryManagerId,OrderHeaderNumber,OrderLineNumber,Inbound Duty,Quantity,Amount,Discount,Cost
0,0,139269,43,222,B2BSUP,71,Finland,20220312,2,1718,WHOLESALE,FI,54,347900,F15404308_CLR000021,F15404308,Hoys pants 6528,Black,XL,5.711450e+12,WOMENSWEAR,Trousers,Trousers,Default,Woven,China,315.0,800.0,113,2,103099,20001,0.12,1.0,294.63,17.71,152.91
1,1,139865,43,222,B2BSUP,71,Finland,20220312,2,1718,WHOLESALE,FI,54,347900,F15404308_CLR000021,F15404308,Hoys pants 6528,Black,XL,5.711450e+12,WOMENSWEAR,Trousers,Trousers,Default,Woven,China,315.0,800.0,113,2,103106,20002,0.12,2.0,589.26,35.34,305.80
2,2,137984,43,222,B2BSUP,20,Belgium,20220312,1,1462,WHOLESALE,BE,54,778,M00012003_CLR000508,M00012003,Kronos o-n ss 273,White mel,S,5.711452e+12,MENSWEAR,Tops,Crew neck,Short sleeve,Jersey,Turkey,90.0,250.0,282,3,103035,20002,0.00,1.0,78.12,0.00,42.41
3,3,195101,43,222,B2BSUP,57,Germany,20220312,2,1231,WHOLESALE,DE-2,54,348547,F19123672_CLR000021,F19123672,Majan ss shirt 9942,Black,L,5.711452e+12,WOMENSWEAR,Shirts,Shirts,Short sleeve,Woven,China,195.0,500.0,96,6,IN0001122,60002,0.12,2.0,379.45,0.00,147.84
4,4,138577,43,222,B2BSUP,57,Germany,20220312,2,1231,WHOLESALE,DE-2,54,348547,F19123672_CLR000021,F19123672,Majan ss shirt 9942,Black,L,5.711452e+12,WOMENSWEAR,Shirts,Shirts,Short sleeve,Woven,China,195.0,500.0,85,6,103113,40003,0.12,2.0,379.45,0.00,162.08


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 826690 entries, 0 to 826689
Data columns (total 37 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Unnamed: 0                826690 non-null  int64  
 1   Id                        826690 non-null  int64  
 2   DimEntityId               826690 non-null  int64  
 3   DimOrderTypeId            826690 non-null  int64  
 4   OrderTypeCode             826690 non-null  object 
 5   DimGeographyId            826690 non-null  int64  
 6   Country                   826690 non-null  object 
 7   DimPostingDateId          826690 non-null  int64  
 8   DimGenderId               826690 non-null  int64  
 9   DimBillToCustomerId       826690 non-null  int64  
 10  CustomerMainType          826690 non-null  object 
 11  CustomerSubGroup          826690 non-null  object 
 12  DimCollectionId           826690 non-null  int64  
 13  DimItemId                 826690 non-null  i

In [8]:
# Handling missing values: we notice missing values for EAN, sub category, sub category 2, quality, Inbound duties
# EAN -  Using DimItemId we have an identifiant of product at a Style x Color x Size level. Using ItemNo2, we have Style x Color level. Using StyleID, we have an identifiant at a style level.
# We conclude we can remove EAN as it isn't bringing more information.
df.drop('EAN', axis=1, inplace=True)

In [9]:
# Sub category and sub category 2
unique_cat = df[['Category','SUB Category','SUB Category2']].drop_duplicates()
unique_cat
# NaNs in the product categories are coming solely from Sunglasses. In order to comply with the rest of the categories, we will replace NaN in SUB Category by 'Sunglasses' and SUB Category2 by 'Default'
df['SUB Category'].fillna('Sunglasses', inplace=True)
df['SUB Category2'].fillna('Default', inplace=True)

C:\Users\MaximeRoux\AppData\Local\Temp\ipykernel_36392\3534567760.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['SUB Category'].fillna('Sunglasses', inplace=True)
C:\Users\MaximeRoux\AppData\Local\Temp\ipykernel_36392\3534567760.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a

In [10]:
# Quality
print(f'There are {df[df['Quality'].isna()]['Style'].nunique()} styles without quality.')
# We think we can replace the values by investigating online what is the fabric.
df[df['Quality'].isna()]['Style'].unique()

There are 45 styles without quality.


array(['Camino t-shirt ss 6024', 'Merkur t-n ls 200',
       'Knud t-shirt st 10379', 'Siffy ls body 10908',
       'Tandy panties 6356', 'Eddie jeans 13026', 'Hugo crew neck 11414',
       'Marilyn bra 6356', 'Siff tee 6202', 'Liss ss gd 3174',
       'Mains tee 5687', 'Kronos o-n ss 273', 'Alice jeans 11364',
       'Alice jeans 10232', 'Liam BX shirt 11246', 'Enno hoodie ls 7057',
       'Nelli ls 9400', 'Holger socks 11517', 'Rory jeans 14031',
       'Nobel ls stripe 3173', 'Hugo hoodie 11414',
       'Kleo turtleneck 11265', 'Ester t-n 265', 'Alaya jeans 11363',
       'Alice jeans 11363', 'Sarai wrap ls 12708', 'Siff tee 13114',
       'Kleo cardigan 11265', 'Edger A blazer 6568',
       'Kleo crew neck 11265', 'Rory jeans 13047', 'Cosmo jeans 11358',
       'Patrick o-n ss 10379', 'Kronos o-n stripe 273',
       'Alaya jeans 10232', 'Trunks 3702', 'Enno zip hoodie 7057',
       'Cosmo jeans 13047', 'Rory jeans 11358', 'Solly tee solid 205',
       'Rory jeans 11005', 'Tom o-n s

In [11]:
df['Quality'].unique()

array(['Woven', 'Jersey', nan, 'Denim', 'Knit', 'Leather', 'Shoes',
       'Other'], dtype=object)

In [12]:
NaN_quality_dict = {'Camino t-shirt ss 6024' : 'Jersey',
                     'Merkur t-n ls 200' : 'Jersey',
                     'Knud t-shirt st 10379' : 'Other', 
                     'Siffy ls body 10908' : 'Other',
                     'Tandy panties 6356' : 'Jersey',
                     'Eddie jeans 13026' : 'Denim',
                     'Hugo crew neck 11414' : 'Jersey',
                     'Marilyn bra 6356' : 'Jersey',
                     'Siff tee 6202' : 'Other',
                     'Liss ss gd 3174' : 'Jersey',
                     'Mains tee 5687' : 'Woven',
                     'Kronos o-n ss 273' : 'Jersey',
                     'Alice jeans 11364' : 'Denim',
                     'Alice jeans 10232' : 'Denim',
                     'Liam BX shirt 11246' : 'Woven', 
                     'Enno hoodie ls 7057' : 'Other',
                     'Nelli ls 9400' : 'Jersey',
                     'Holger socks 11517' : 'Knit',
                     'Rory jeans 14031' : 'Denim',
                     'Nobel ls stripe 3173' : 'Jersey', 
                     'Hugo hoodie 11414' : 'Other',
                     'Kleo turtleneck 11265' : 'Knit', 
                     'Ester t-n 265' : 'Jersey', 
                     'Alaya jeans 11363' : 'Denim',
                     'Alice jeans 11363' : 'Denim',
                     'Sarai wrap ls 12708' : 'Other',
                     'Siff tee 13114' : 'Jersey',
                     'Kleo cardigan 11265' : 'Knit',
                     'Edger A blazer 6568' : 'Other',
                     'Kleo crew neck 11265' : 'Other',
                     'Rory jeans 13047' : 'Denim',
                     'Cosmo jeans 11358' : 'Denim',
                     'Patrick o-n ss 10379' : 'Jersey',
                     'Kronos o-n stripe 273' : 'Jersey',
                     'Alaya jeans 10232' : 'Denim',
                     'Trunks 3702' : 'Other',
                     'Enno zip hoodie 7057' : 'Other',
                     'Cosmo jeans 13047' : 'Denim',
                     'Rory jeans 11358' : 'Denim',
                     'Solly tee solid 205' : 'Jersey',
                     'Rory jeans 11005' : 'Denim',
                     'Tom o-n ss 10076' : 'Other',
                     'Adelina jeans 14145' : 'Other',
                     'Amie ls 2085' : 'Jersey',
                     'Sunglasses Display' : 'Other'}

In [13]:
for i in NaN_quality_dict:
    df.loc[df['Style']==i,['Quality']]=NaN_quality_dict[i]

In [14]:
df['Quality'].unique()

array(['Woven', 'Jersey', 'Denim', 'Other', 'Knit', 'Leather', 'Shoes'],
      dtype=object)

In [15]:
# Inbound duties - We will not use this feature
df.drop('Inbound Duty', axis=1, inplace=True)

In [16]:
df.info()
# No more NaN values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 826690 entries, 0 to 826689
Data columns (total 35 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Unnamed: 0                826690 non-null  int64  
 1   Id                        826690 non-null  int64  
 2   DimEntityId               826690 non-null  int64  
 3   DimOrderTypeId            826690 non-null  int64  
 4   OrderTypeCode             826690 non-null  object 
 5   DimGeographyId            826690 non-null  int64  
 6   Country                   826690 non-null  object 
 7   DimPostingDateId          826690 non-null  int64  
 8   DimGenderId               826690 non-null  int64  
 9   DimBillToCustomerId       826690 non-null  int64  
 10  CustomerMainType          826690 non-null  object 
 11  CustomerSubGroup          826690 non-null  object 
 12  DimCollectionId           826690 non-null  int64  
 13  DimItemId                 826690 non-null  i

In [17]:
# Remove the non commercial subgroups: internal channels
subgrouptoremove = ['X_PR', 'X_SHOWROOM', 'X_OTHER', 'X_IC','Ungrouped']
dfsales = df[~df['CustomerSubGroup'].isin(subgrouptoremove)]

As explained in the document, for business reason, we have decided to focus on the product hierarchy rather than the geographical hierarchy. Hence, we decide to remove all the information related to the customer hierarchy.

In [18]:
dfsales = dfsales.drop(['DimOrderTypeId', 'OrderTypeCode', 'DimGeographyId', 'Country', 'DimBillToCustomerId', 'CustomerMainType', 'CustomerSubGroup',
                      'DimOrderSalespersonId', 'DimOrderCountryManagerId'], axis=1)

In [19]:
# The following columns are useless: Unnamed: 0 - SQL extract index, Id - Id coming from the initial table, DimEntityId - Unique value corresponding to the brand Samsøe Samsøe, DimCollectionId - Unique value corresponding
# to the Basic collection
dfsales.drop(['Unnamed: 0', 'Id', 'DimEntityId', 'DimCollectionId'], axis=1, inplace=True)

In [20]:
# Create timeseries and change idex
dfsales['DimPostingDateId'] = pd.to_datetime(dfsales['DimPostingDateId'],format='%Y%m%d')
dfsales.set_index("DimPostingDateId", inplace=True)

In [21]:
dfsales.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 818375 entries, 2022-03-12 to 2024-11-18
Data columns (total 21 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   DimGenderId        818375 non-null  int64  
 1   DimItemId          818375 non-null  int64  
 2   ItemNo2            818375 non-null  object 
 3   StyleID            818375 non-null  object 
 4   Style              818375 non-null  object 
 5   Color              818375 non-null  object 
 6   Size               818375 non-null  object 
 7   Gender             818375 non-null  object 
 8   Category           818375 non-null  object 
 9   SUB Category       818375 non-null  object 
 10  SUB Category2      818375 non-null  object 
 11  Quality            818375 non-null  object 
 12  Origin Country     818375 non-null  object 
 13  WHSSalesPriceDKK   818375 non-null  float64
 14  RRSalesPriceDKK    818375 non-null  float64
 15  OrderHeaderNumber  818375 non-null 

In [22]:
dfsales.to_csv("data/cleaned_data.csv")

## 2. Train-Test split

In [23]:
dfsales.index = pd.to_datetime(dfsales.index, format="%Y-%m-%d")

In [24]:
df = dfsales.groupby('ItemNo2').resample('W')['Quantity'].sum().reset_index()

In [25]:
df = df.reindex(columns=['DimPostingDateId', 'ItemNo2', 'Quantity'])

In [26]:
ts = df

In [27]:
ts.columns=['ds', 'unique_id', 'y']

In [28]:
ts = ts.reindex(columns=["unique_id", "ds", "y"])

In [29]:
# Add zeros to have a regular period
ts0 = ts.pivot(index='ds', columns='unique_id', values='y')
ts0 = ts0.fillna(0)

In [30]:
ts0 = ts0.T
ts0 = ts0.reset_index()

In [31]:
ts0

ds,unique_id,2022-03-13 00:00:00,2022-03-20 00:00:00,2022-03-27 00:00:00,2022-04-03 00:00:00,2022-04-10 00:00:00,2022-04-17 00:00:00,2022-04-24 00:00:00,2022-05-01 00:00:00,2022-05-08 00:00:00,2022-05-15 00:00:00,2022-05-22 00:00:00,2022-05-29 00:00:00,2022-06-05 00:00:00,2022-06-12 00:00:00,2022-06-19 00:00:00,2022-06-26 00:00:00,2022-07-03 00:00:00,2022-07-10 00:00:00,2022-07-17 00:00:00,2022-07-24 00:00:00,2022-07-31 00:00:00,2022-08-07 00:00:00,2022-08-14 00:00:00,2022-08-21 00:00:00,2022-08-28 00:00:00,2022-09-04 00:00:00,2022-09-11 00:00:00,2022-09-18 00:00:00,2022-09-25 00:00:00,2022-10-02 00:00:00,2022-10-09 00:00:00,2022-10-16 00:00:00,2022-10-23 00:00:00,2022-10-30 00:00:00,2022-11-06 00:00:00,2022-11-13 00:00:00,2022-11-20 00:00:00,2022-11-27 00:00:00,2022-12-04 00:00:00,2022-12-11 00:00:00,2022-12-18 00:00:00,2022-12-25 00:00:00,2023-01-01 00:00:00,2023-01-08 00:00:00,2023-01-15 00:00:00,2023-01-22 00:00:00,2023-01-29 00:00:00,2023-02-05 00:00:00,2023-02-12 00:00:00,2023-02-19 00:00:00,2023-02-26 00:00:00,2023-03-05 00:00:00,2023-03-12 00:00:00,2023-03-19 00:00:00,2023-03-26 00:00:00,2023-04-02 00:00:00,2023-04-09 00:00:00,2023-04-16 00:00:00,2023-04-23 00:00:00,2023-04-30 00:00:00,2023-05-07 00:00:00,2023-05-14 00:00:00,2023-05-21 00:00:00,2023-05-28 00:00:00,2023-06-04 00:00:00,2023-06-11 00:00:00,2023-06-18 00:00:00,2023-06-25 00:00:00,2023-07-02 00:00:00,2023-07-09 00:00:00,2023-07-16 00:00:00,2023-07-23 00:00:00,2023-07-30 00:00:00,2023-08-06 00:00:00,2023-08-13 00:00:00,2023-08-20 00:00:00,2023-08-27 00:00:00,2023-09-03 00:00:00,2023-09-10 00:00:00,2023-09-17 00:00:00,2023-09-24 00:00:00,2023-10-01 00:00:00,2023-10-08 00:00:00,2023-10-15 00:00:00,2023-10-22 00:00:00,2023-10-29 00:00:00,2023-11-05 00:00:00,2023-11-12 00:00:00,2023-11-19 00:00:00,2023-11-26 00:00:00,2023-12-03 00:00:00,2023-12-10 00:00:00,2023-12-17 00:00:00,2023-12-24 00:00:00,2023-12-31 00:00:00,2024-01-07 00:00:00,2024-01-14 00:00:00,2024-01-21 00:00:00,2024-01-28 00:00:00,2024-02-04 00:00:00,2024-02-11 00:00:00,2024-02-18 00:00:00,2024-02-25 00:00:00,2024-03-03 00:00:00,2024-03-10 00:00:00,2024-03-17 00:00:00,2024-03-24 00:00:00,2024-03-31 00:00:00,2024-04-07 00:00:00,2024-04-14 00:00:00,2024-04-21 00:00:00,2024-04-28 00:00:00,2024-05-05 00:00:00,2024-05-12 00:00:00,2024-05-19 00:00:00,2024-05-26 00:00:00,2024-06-02 00:00:00,2024-06-09 00:00:00,2024-06-16 00:00:00,2024-06-23 00:00:00,2024-06-30 00:00:00,2024-07-07 00:00:00,2024-07-14 00:00:00,2024-07-21 00:00:00,2024-07-28 00:00:00,2024-08-04 00:00:00,2024-08-11 00:00:00,2024-08-18 00:00:00,2024-08-25 00:00:00,2024-09-01 00:00:00,2024-09-08 00:00:00,2024-09-15 00:00:00,2024-09-22 00:00:00,2024-09-29 00:00:00,2024-10-06 00:00:00,2024-10-13 00:00:00,2024-10-20 00:00:00,2024-10-27 00:00:00,2024-11-03 00:00:00,2024-11-10 00:00:00,2024-11-17 00:00:00,2024-11-24 00:00:00
0,F00001015_CLR000021,0.0,66.0,92.0,90.0,70.0,78.0,93.0,58.0,62.0,68.0,69.0,147.0,59.0,93.0,62.0,61.0,69.0,3.0,101.0,102.0,26.0,41.0,21.0,32.0,31.0,43.0,55.0,42.0,53.0,95.0,49.0,84.0,27.0,10.0,92.0,38.0,83.0,94.0,87.0,50.0,111.0,136.0,36.0,57.0,15.0,38.0,15.0,36.0,10.0,49.0,34.0,60.0,144.0,40.0,48.0,53.0,52.0,67.0,50.0,76.0,71.0,40.0,37.0,84.0,72.0,90.0,75.0,53.0,34.0,71.0,72.0,85.0,64.0,87.0,57.0,59.0,49.0,35.0,44.0,48.0,45.0,73.0,87.0,65.0,85.0,45.0,33.0,47.0,46.0,69.0,82.0,88.0,98.0,95.0,60.0,64.0,40.0,33.0,37.0,84.0,50.0,58.0,58.0,39.0,69.0,45.0,64.0,26.0,67.0,76.0,89.0,56.0,83.0,43.0,76.0,31.0,28.0,82.0,63.0,50.0,49.0,51.0,55.0,41.0,46.0,29.0,50.0,67.0,31.0,60.0,31.0,26.0,53.0,65.0,53.0,28.0,55.0,42.0,17.0,48.0,50.0,0.0
1,F00001015_CLR000023,0.0,49.0,54.0,50.0,45.0,87.0,49.0,54.0,73.0,49.0,68.0,97.0,42.0,47.0,64.0,39.0,48.0,2.0,63.0,91.0,33.0,20.0,16.0,27.0,30.0,17.0,33.0,52.0,34.0,54.0,27.0,47.0,8.0,6.0,42.0,17.0,39.0,13.0,7.0,22.0,48.0,32.0,14.0,8.0,8.0,23.0,10.0,16.0,6.0,22.0,23.0,37.0,83.0,22.0,39.0,23.0,55.0,31.0,22.0,44.0,47.0,32.0,18.0,66.0,77.0,47.0,29.0,36.0,20.0,30.0,34.0,18.0,9.0,9.0,11.0,25.0,9.0,170.0,21.0,21.0,

In [32]:
ts0 = ts0.melt(id_vars=["unique_id"], var_name='ds', value_name="y")

In [33]:
ts0.ds = pd.to_datetime(ts0.ds, format="%Y-%m-%d")

In [34]:
ts0.to_csv("data/timeseries.csv")

In [39]:
timesplit = ts0.ds.max() - timedelta(weeks=8)

In [43]:
df_train = ts0[ts0['ds'] < timesplit]
df_test = ts0[ts0['ds'] >= timesplit]

train_perc = np.round(100 * df_train.shape[0] / ts0.shape[0])
test_perc = np.round(100 * df_test.shape[0] / ts0.shape[0])
print(f"The dataset is divided as follow: {train_perc}% of the data used for training and {test_perc}% for testing")

The dataset is divided as follow: 94.0% of the data used for training and 6.0% for testing


In [44]:
df_train.to_csv("data/train.csv")
df_test.to_csv ("data/test.csv")